# Numpy Memory Array for Fast Image Loading

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.append(repo_root)

import cv2
import numpy as np
from PIL import Image

from model.memory_map import MemMapReader, MemMapWriter

## Writer

In [ ]:
source_path = Path(f'{repo_root}/outputs/speed_test')
image_paths = list(source_path.glob('*.png'))
mm_writer = MemMapWriter(image_paths, 'outputs/mm/mem_map.dat')
mm_writer.write()
print(mm_writer)

Saving memmap: 100%|██████████| 476/476 [00:00<00:00, 1850.03it/s]

MemMap with 476 images:
- Output file: outputs/mm/mem_map.dat
- Resize: (256, 256)
- dtype: uint8
- func: crop_resize_driver


## Reader

In [7]:
mm_reader = MemMapReader('outputs/mm/mem_map.dat', (256, 256))
print(mm_reader)

MemMap with 476 images:
- File: outputs/mm/mem_map.dat
- Shape: (256, 256)


In [8]:
len(mm_reader)

476

In [9]:
mm_reader[len(mm_reader) - 1].shape

(256, 256)

### Memory Array

Memory array creates **read-only** numpy arrays.

In [10]:
%%timeit -n 10

for img in mm_reader:
    x = img.shape

455 μs ± 93.9 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit -n 10

for img in mm_reader:
    x = np.copy(img).shape

1.76 ms ± 306 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit -n 10

for imgs in mm_reader.iter_windows(4):
    x = (len(imgs), imgs[0].shape)

2.02 ms ± 304 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit -n 10

for imgs in mm_reader.iter_windows_mut(4):
    x = (len(imgs), imgs[0].shape)

6.74 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Pillow

In [14]:
%%timeit -n 10

for img_path in image_paths:
    x = np.array(Image.open(img_path)).shape

109 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit -n 10

for i in range(0, len(image_paths), 4):
    xs = [np.array(Image.open(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

108 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### OpenCV

In [16]:
%%timeit -n 10

for img_path in image_paths:
    x = cv2.imread(str(img_path)).shape

102 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit -n 10

for i in range(0, len(image_paths), 4):
    xs = [cv2.imread(str(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

103 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Benchmark Result

We used [`timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) magic command to measure the time.

Memory array is up-to ~20 times faster than Pillow or OpenCV.